In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# 開始年
start_y=2014
# dfにするファイル名
files=['tp20151211-01_1.xls',
'tp20151211-01_3.xls',
'tp20151211-01_1.xls',
'tp20151211-01_3.xls',
'tp20180314-01_1.xls',
'tp20171208-01_3.xls',
'tp20180314-01_1.xls',
'tp20171208-01_3.xls',
'tp20190904-01_01.xls',
'tp20190904-01_03.xls',
'tp20200122-01_01.xls',
'tp20200122-01_03.xls',
'tp20210218-01_01.xlsx',
'tp20210218-01_03.xlsx',
'tp20220201-01_01.xlsx',
'tp20211210-01_03.xlsx']
for i,name in enumerate(files):
    print(start_y+i//2,name)

2014 tp20151211-01_1.xls
2014 tp20151211-01_3.xls
2015 tp20151211-01_1.xls
2015 tp20151211-01_3.xls
2016 tp20180314-01_1.xls
2016 tp20171208-01_3.xls
2017 tp20180314-01_1.xls
2017 tp20171208-01_3.xls
2018 tp20190904-01_01.xls
2018 tp20190904-01_03.xls
2019 tp20200122-01_01.xls
2019 tp20200122-01_03.xls
2020 tp20210218-01_01.xlsx
2020 tp20210218-01_03.xlsx
2021 tp20220201-01_01.xlsx
2021 tp20211210-01_03.xlsx


なぜ複雑な処理をするかというと、年度ごとに生産企業名が変わっていたり、年度途中で追加、廃止されたりしていて、ndbのほうとラグが出ているために、うまくjoinできない場合があるから

In [3]:
firm_data=pd.DataFrame()
for i,name in enumerate(files):
    print(start_y+i//2,name)
    data_i=pd.read_excel(name)
    data_i['year']=start_y+i//2
    firm_data = pd.concat([firm_data,data_i],axis=0,ignore_index=True)
    this_year = start_y+i//2
    last_year = (start_y+i//2)-1
    # 1年に1回のみ実行
    if i%2==1:
        # 現在のDataFrameから年とIDの組み合わせを取得
        firm_data_two=firm_data[firm_data['year']>=(last_year)]
        existing_combinations = set(zip(firm_data_two['year'], firm_data_two["薬価基準収載医薬品コード"]))

        # 去年に、今年のIDがない場合、今年のIDを持つ行をコピー、昨年に対応する行を作成
        new_rows = []
        for this_year, id in existing_combinations:
            if (last_year, id) not in existing_combinations:
                this_year_row = firm_data[(firm_data['year'] == this_year) & (firm_data["薬価基準収載医薬品コード"] == id)]
                new_row = this_year_row.copy()
                new_row['year'] = last_year
                new_rows.append(new_row)
        if this_year==2019 or this_year==2020:
            # 今年に、last yearのIDがない場合、last yearのIDを持つ行をコピーしthis yearに対応する行を作成
            for last_year, id in existing_combinations:
                if (this_year, id) not in existing_combinations:
                    last_year_row = firm_data[(firm_data['year'] == last_year) & (firm_data["薬価基準収載医薬品コード"] == id)]
                    new_row = last_year_row.copy()
                    new_row['year'] = this_year
                    new_rows.append(new_row)
        # 新しい行を現在のDataFrameに追加
        firm_data = pd.concat([firm_data] + new_rows,axis=0, ignore_index=True)
    

2014 tp20151211-01_1.xls
2014 tp20151211-01_3.xls
2015 tp20151211-01_1.xls
2015 tp20151211-01_3.xls
2016 tp20180314-01_1.xls
2016 tp20171208-01_3.xls
2017 tp20180314-01_1.xls
2017 tp20171208-01_3.xls
2018 tp20190904-01_01.xls
2018 tp20190904-01_03.xls
2019 tp20200122-01_01.xls
2019 tp20200122-01_03.xls
2020 tp20210218-01_01.xlsx
2020 tp20210218-01_03.xlsx
2021 tp20220201-01_01.xlsx
2021 tp20211210-01_03.xlsx


In [4]:
firm_data=firm_data[firm_data["メーカー名"].notna()]
firm_data.shape

(111256, 21)

大正薬品工業は名前を変えた

In [5]:
firm_data["year"]

1         2014
2         2014
3         2014
4         2014
5         2014
          ... 
121776    2020
121778    2020
121779    2020
121780    2020
121781    2020
Name: year, Length: 111256, dtype: int64

In [6]:
dup=firm_data[firm_data.duplicated(["薬価基準収載医薬品コード","year"])]
#同じ年で名前が変わっている? 2023まで入れればね
for id in set(dup["薬価基準収載医薬品コード"]):
    df_id=dup[dup["薬価基準収載医薬品コード"]==id]
    year=set(df_id["year"])
    maker=set(df_id["メーカー名"])
    if len(maker)>1:
        print(id,year)
        print(maker)

In [7]:
len(set(firm_data["メーカー名"])),len(set(firm_data["薬価基準収載医薬品コード"])),sum(firm_data["メーカー名"].isna())

(337, 16386, 0)

In [8]:
print(set(firm_data["メーカー名"]))

{'キッセイ薬品工業', 'シオノケミカル', '昭和電工', 'Ｍｅ\u3000ファルマ', 'レコルダティ・レア・ディジーズ・ジャパン', '日本粉末薬品', 'グラクソ・スミスクライン', '鳥居薬品', 'サンファーマ', '太虎精堂製薬', '辰巳化学', '恵美須薬品化工', 'エムジーファーマ', '日本家庭用塩', 'オルガノン', 'ガルデルマ', 'ニプロファーマ', 'マルホ', '化研生薬', '日新製薬（山形）', 'トライックス', '日医工サノフィ', '日本たばこ産業', 'ウチダ和漢薬', '日産化学', '東亜薬品', '昭和薬品化工', '小島漢方', '大協薬品工業', '廣貫堂', '福地製薬', 'ゾンネボード製薬', 'アトルファーマ', 'ライオン', '堀井薬品工業', 'サンノーバ', 'セルジーン', 'オオサキメディカル', '高杉製薬', 'ＩＬＳ', '森下仁丹', '富士フイルムファーマ', '日興製薬', '協和発酵キリン', 'トクホン', '高砂薬業', '塩野義製薬', '金田直隆商店', 'コーアバイオテックベイ', 'ブリストル・マイヤーズ\u3000スクイブ', '大正製薬', '三和化学研究所', '東レ', '日本エア・リキード', 'ＤＳファーマバイオメディカル', 'ヤクハン製薬', '健栄製薬', '日医工岐阜工場', '日本イーライリリー', 'Ｈｕｙａ\u3000Ｊａｐａｎ', '三全製薬', '藤永製薬', '東レ・メディカル', '旭化成ファーマ', 'ジェーピーエス製薬', '大晃生薬', 'アールテック・ウエノ', '大正薬品工業', '佐藤製薬', '杏林製薬', '大塚製薬工場', 'ユーシービージャパン', 'ポーラファルマ', '日本アルコン', 'ジェイドルフ製薬', 'テバ製薬', '持田製薬', 'メルクセローノ', '小野薬品工業', '帝人ファーマ', 'エイエムオー・ジャパン', 'ブリストル・マイヤーズ', '東菱薬品工業', '皇漢堂製薬', '味の素製薬', '大峰堂薬品工業', '化学及血清療法研究所', 'ファイザー', 'バイエル薬品', 'ミノファーゲン製薬', '帝國漢方製薬', '三和生薬', '安西メディカル', 'シー・エイチ・

In [9]:
data_ndb=pd.read_csv("../merged/len3_ndb_blp_DN.csv",encoding="shift-jis",index_col=0)
# data_n=data_n[data_n["year"]>=2018]
print("initial",len(set(data_ndb["薬価基準収載医薬品コード"])))

initial 2671


In [10]:
data_ndb.columns

Index(['薬効分類', '薬効分類名称', '医薬品コード', '医薬品名', '薬価基準収載医薬品コード', '薬価', '後発品区分', '総計',
       'year', 'in_hospital',
       ...
       'id_625.0', 'id_629.0', 'id_799.0', 'q_share', 'r_share',
       'generic_share_q', 'generic_share_r', 'lag_generic_share_q',
       'mean_price_g', 'mean_price_b'],
      dtype='object', length=112)

In [11]:
len(set(data_ndb["薬価基準収載医薬品コード"]) - set(firm_data["薬価基準収載医薬品コード"]))

207

In [12]:
data_ndb.shape

(25798, 112)

In [13]:
data_ndb.columns

Index(['薬効分類', '薬効分類名称', '医薬品コード', '医薬品名', '薬価基準収載医薬品コード', '薬価', '後発品区分', '総計',
       'year', 'in_hospital',
       ...
       'id_625.0', 'id_629.0', 'id_799.0', 'q_share', 'r_share',
       'generic_share_q', 'generic_share_r', 'lag_generic_share_q',
       'mean_price_g', 'mean_price_b'],
      dtype='object', length=112)

In [14]:
data_ndb.duplicated(subset=['year', "医薬品コード","in_hospital"],keep=False).sum()

0

yearを最初は入れない

In [15]:
# data_tmp=data_ndb.merge(firm_data,on=["薬価基準収載医薬品コード"],how="left",indicator=True)
# condition = (data_tmp['year_x'] != data_tmp['year_y']) & data_tmp.duplicated(subset=['year_x', "医薬品コード","in_hospital"], keep=False)
# # condition =  data_tmp.duplicated(subset=['year_x', "医薬品コード","in_hospital"],keep=False)
# data_tmp = data_tmp[~condition]
# data_tmp=data_ndb.merge(firm_data,on=["薬価基準収載医薬品コード"],how="left",indicator=True).query('_merge == "left_only"')
# print(data_tmp.shape,data_left_only.shape)
# print(data_tmp["_merge"].value_counts())

In [16]:
# duplicated かつ一つもyearが一致しないものがある?
# condition =  data_tmp.duplicated(subset=['year_x', "医薬品コード","in_hospital"],keep=False)
# data_tmp = data_tmp[condition]
# data_tmp[["in_hospital","医薬品コード","year_x"]].value_counts()
# data_tmp2=data_tmp[~(data_tmp['year_x'] != data_tmp['year_y'])]
# data_tmp2[["in_hospital","医薬品コード","year_x"]].value_counts()
# set(map(tuple,data_tmp[["in_hospital","医薬品名","year_x"]].to_records(index=False)))-set(map(tuple,data_tmp2[["in_hospital","医薬品名","year_x"]].to_records(index=False)))
# len(set(data_data_tmp["薬価基準収載医薬品コード"]))

In [17]:
value_counts = firm_data.groupby("薬価基準収載医薬品コード")['メーカー名'].nunique()

# makerの種類数が1つの行を抽出する
unique_maker_rows = firm_data[firm_data["薬価基準収載医薬品コード"].map(value_counts) == 1]
code2maker = unique_maker_rows.set_index('薬価基準収載医薬品コード')['メーカー名'].to_dict()
unique_maker_rows
code2maker

{'1121001X1085': '中北薬品',
 '1121001X1131': '山善製薬',
 '1121001X1204': '日本新薬',
 '1121001X1212': '吉田製薬',
 '1121001X1220': '日本ジェネリック',
 '1124001F1030': '共和薬品工業',
 '1124001F2037': '共和薬品工業',
 '1124002M1042': '協和発酵キリン',
 '1124002M2022': '共和薬品工業',
 '1124002M2049': '協和発酵キリン',
 '1124003C1106': '辰巳化学',
 '1124003C1114': '辰巳化学',
 '1124003F2176': '辰巳化学',
 '1124003F2184': 'テバ製薬',
 '1124003F2249': '東和薬品',
 '1124003F2257': 'コーアイセイ',
 '1124003F2265': '日本ジェネリック',
 '1124003F2273': '鶴原製薬',
 '1124003F2281': 'コーアイセイ',
 '1124003F2303': '辰巳化学',
 '1124003F3156': '日本ジェネリック',
 '1124004F1042': '大日本住友製薬',
 '1124004F2030': '大日本住友製薬',
 '1124007F1020': 'ファイザー',
 '1124007F1038': '富士薬品',
 '1124007F1046': 'サンノーバ',
 '1124007F1070': '大興製薬',
 '1124007F1089': '日新製薬（山形）',
 '1124007F1097': '辰巳化学',
 '1124007F1100': '長生堂製薬',
 '1124007F1119': '小林化工',
 '1124007F1135': '日医工',
 '1124007F1143': '長生堂製薬',
 '1124007F1151': '小林化工',
 '1124007F1160': '日新製薬（山形）',
 '1124007F2026': 'ファイザー',
 '1124007F2042': '日新製薬（山形）',
 '1124007F2050': '寿製薬',
 

year を入れてもうまくいく

In [18]:
data_tmp=data_ndb.merge(firm_data,on=["薬価基準収載医薬品コード","year"],how="left",indicator=True)
data_tmp.loc[data_tmp['_merge'] == 'left_only', 'メーカー名'] = data_tmp.loc[data_tmp['_merge'] == 'left_only','薬価基準収載医薬品コード'].map(code2maker)
# data_left_only=data_ndb.merge(firm_data,on=["薬価基準収載医薬品コード","year"],how="left",indicator=True).query('_merge == "left_only"')
print(data_tmp.shape,data_tmp.shape)
print(data_tmp["_merge"].value_counts())

(25798, 132) (25798, 132)
both          21962
left_only      3836
right_only        0
Name: _merge, dtype: int64


In [19]:
dups=data_tmp[data_tmp.duplicated(subset=["year","医薬品コード","in_hospital"],keep=False)]
print(dups[["year","医薬品コード","メーカー名","in_hospital"]])

Empty DataFrame
Columns: [year, 医薬品コード, メーカー名, in_hospital]
Index: []


In [20]:
# data_left_only.loc[data_left_only["メーカー名"].isna(),"メーカー名"]=data_left_only.loc[data_left_only["メーカー名"].isna(),"医薬品名"].map(map_dict)
# len(set(data_left_only["薬価基準収載医薬品コード"])-set(firm_data["薬価基準収載医薬品コード"]))
# len(set(data_left_only["薬価基準収載医薬品コード"])-unique_set),len(set(data_left_only["薬価基準収載医薬品コード"])),len(set(data_left_only["医薬品名"]))

In [21]:
# data_tmp.loc[data_tmp["メーカー名"].isna(),"メーカー名"]=data_tmp.loc[data_tmp["メーカー名"].isna(),"薬価基準収載医薬品コード"].map(map_dict)
data_rest=data_tmp.loc[data_tmp["メーカー名"].isna()]
print(len(set(data_rest["薬価基準収載医薬品コード"])-set(firm_data["薬価基準収載医薬品コード"])))
len(set(data_rest["薬価基準収載医薬品コード"])),len(set(data_tmp["薬価基準収載医薬品コード"])),len(set(data_rest["医薬品名"]))

207


(208, 2671, 806)

In [22]:
data_tmp.to_csv("../merged/len3_ndb_blp_DN_firm.csv",encoding="utf-8")